In [1]:
from config import *
from selenium import webdriver
import json
import sys
import datetime
import time

In [2]:
def write_out(message):
     with open('Bartley_Booker_Logs.txt','a+') as f:
                f.write(datetime.datetime.today().strftime('%Y-%m-%d:%H:%M:%S') + ": "+ message + "\n")
def verify_bookings():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome('./chromedriver_win32/chromedriver.exe',options=chrome_options)
#     driver = webdriver.Chrome('./chromedriver_win32/chromedriver.exe')
    driver.get('http://bartleyrid.gotdns.com:8081/booking/mybookings.aspx?')
    driver.find_element_by_xpath('//*[@id="txtUser"]').send_keys(keys["user_id"]) #User
    driver.find_element_by_xpath('//*[@id="txtPassword"]').send_keys(keys["password"]) #Password
    driver.find_element_by_xpath('//*[@id="PageContentArea"]/form[1]/table/tbody/tr/td/table/tbody/tr/td/input[3]').click() #Submit
    
    book_date = datetime.datetime.today() + datetime.timedelta(days=14)#use 14 when testing 15 when executing
    book_date_str = book_date.strftime('%d/%m/%Y')
    booking_list = []
    booking_info = {}    
    id_list=[]
    tbl = driver.find_element_by_id("Table1")
    ids = tbl.find_elements_by_xpath('//*[@id]')
    #Get a list of Ids from all items with IDs
    elements = [x for x in ids if x.get_attribute('id').startswith('ID_')]
    for element in elements:
        sub_element = element.find_elements_by_class_name("form3")
        booking_date = sub_element[0].text
        book_time = sub_element[1].text
        facility = sub_element[2].text

        booking_info = {'date': booking_date,
                        'time' : book_time,
                        'facility': facility}

        booking_list.append(booking_info)

    for booking in booking_list:
        if booking['date'] == book_date_str:
            write_out(json.dumps(booking))
        else:
            continue
    driver.quit()